    0000000_16_230203_003_combine_map_7group_k20man_before
        
    data : 0210823_socar_zone(soacr).csv, happy_house_df1.csv(행복주택), k_university.csv(대학교),
           subway_total_3.csv(지하철),    k_apt_df.csv(아파트)         , k_bus_info.csv(버스),
           subway_transfer.csv(환승역)
           
    display : 230203_combine_7group_k20man.html
       
    목차 
        1) package load
        2) socar data load : 전체 -> 경기도
        3) 행복주택 Data load
        4) 대학교 Data load
        5) 지하철 Data load 
        6) 아파트 data load 
        7) 버스 data load  
        8) 환승역 data load 
        9) 시각화 
           - base map 생성 -> 경기도 경계 표시 -> 20-30 밀도 시각화 -> 레이어 적용을 위한 그룹 만들기
           - 쏘카존 -> 행복주택 -> 대학교 -> 아파트 -> 지하철 -> 버스 -> 환승역
        10) map save

In [1]:
#1) package load
import requests
import json
import os
import math
import time
import numpy as np
import pandas as pd
import geopandas as gpd
import pyproj
import folium
import matplotlib.pyplot as plt
import seaborn as sns

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap

import warnings
warnings.filterwarnings(action='ignore') 

print(pd.__version__)
print(gpd.__version__)
print(folium.__version__)

1.3.3
0.10.1
0.12.1


In [2]:
#2) socar data load : 전체 -> 경기도

path = '../002_data_dir/004_node_data/20210823_socar_zone.csv'

socar_zone_info_df = pd.read_csv(path)
k_socar_zone_df = socar_zone_info_df[socar_zone_info_df['region1'] == '경기도']

In [3]:
#3) 행복주택 Data load
path = '../002_data_dir/003_search_data/happy_house_df1.csv'
happy_house_df = pd.read_csv(path)
happy_house_df.head()                       

,지역,단지,세대수,주소,lng,lat
0,고양시 덕양구,삼송11단지,832,경기도 고양시 덕양구 원흥1로 25,126.865491,37.650992
1,고양시 덕양구,고양지축 A-1블록 신혼희망타운 행복주택,250,경기도 고양시 덕양구 오부자로 99,126.921037,37.653331
2,고양시 덕양구,고양지축 A-4BL 행복주택,473,경기도 고양시 덕양구 오부자로 43,126.914946,37.654038
3,고양시 덕양구,고양삼송 LH원흥마을13단지,947,경기도 고양시 덕양구 원흥1로 35,126.868888,37.652631
4,고양시 덕양구,서정마을LH12단지,276,경기도 고양시 덕양구 서정마을2로 17,126.847648,37.621215


In [4]:
#4) 대학교 Data load
path = '../002_data_dir/003_search_data/k_university.csv'
k_university_df = pd.read_csv(path)
k_university_df.head()

,학교구분,학교코드,학교명,본분교,학제,지역,설립구분,관련법령,법인명,학교상태,...,우편번호,학교개교일,학교홈페이지,총장(대학원장)명,학교대표\r\n번호,학교대표\r\n팩스번호,x,y,lng,lat
0,대학,37.0,한경대학교,본교,대학교,경기,국립,고등교육법,해당없음,기존,...,17579,2012-03-01,www.hknu.ac.kr,이원희,031-670-5114,031-670-5469,978801.712957,1.890391e+06,127.261716,37.011780
1,대학,46.0,가톨릭대학교,본교,대학교,경기,사립,고등교육법,가톨릭학원,기존,...,14662,1995-03-01,www.catholic.ac.kr,원종철,02-2164-4114,02-2164-4754,938317.711386,1.943204e+06,126.802293,37.486022
2,대학,51.0,강남대학교,본교,대학교,경기,사립,고등교육법,강남학원,기존,...,16979,1948-08-12,www.kangnam.ac.kr,윤신일,031-280-3500,031-280-3173,967379.084547,1.919514e+06,127.132051,37.273974
3,대학,56.0,경기대학교,본교,대학교,경기,사립,고등교육법,경기학원,기존,...,16227,1947-11-08,www.kyonggi.ac.kr,전준철(직무대행),031-249-9114,031-243-2194,958688.169183,1.922509e+06,127.033857,37.300618
4,대학,63.0,가천대학교,본교,대학교,경기,사립,고등교육법,가천학원,기존,...,13120,2012-03-01,www.gachon.ac.kr,이길여,031-750-5114,031-753-8828,967163.380457,1.939303e+06,127.128740,37.452335


In [5]:
#5) 지하철 Data load 

path = '../002_data_dir/003_search_data/subway_total_3.csv'
k_subway_info_df = pd.read_csv(path)
k_subway_info_df.head()

,철도운영기관명,선명,역명,지번주소,도로명주소,x,y,lng,lat
0,코레일,1호선,가능,경기도 의정부시 가능동 197-1,경기도 의정부시 평화로 633,959850.319275,1.972172e+06,127.044255,37.748305
1,코레일,1호선,가산디지털단지,서울특별시 금천구 가산동 468-4,서울특별시 금천구 벚꽃로 309,945373.187085,1.942741e+06,126.882129,37.482290
2,코레일,1호선,간석,인천광역시 남동구 간석4동 762번지,인천광역시 남동구 석정로 522-14,928718.830404,1.940941e+06,126.693945,37.464929
3,코레일,1호선,개봉,서울특별시 구로구 개보동 415,서울특별시 구로구 경인로40길 47,943348.199060,1.944111e+06,126.859121,37.494522
4,코레일,1호선,관악,경기도 안양시 만악구 석수동 101-16,경기도 안양시 만안구 경수대로1273번길 46,947714.862265,1.935684e+06,126.909115,37.418821


In [6]:
#6) 아파트 data load   
path = '../002_data_dir/003_search_data/k_apt_df.csv'
k_apt_df = pd.read_csv(path)
k_apt_df = k_apt_df.dropna(axis=0)
# k_apt_df_notnull = k_apt_df[['공동주택명정보','lng', 'lat']].dropna()
# 위도,경도 data만 drop하면 좋음  ... 958개
k_apt_df['lng'] = k_apt_df['lng'].astype(float)
k_apt_df['lat'] = k_apt_df['lat'].astype(float)
k_apt_df = k_apt_df.reset_index(drop=True)
k_apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr,x,y,lng,lat
0,안양시,DY town,호계3동,971-1,2013-01-03,9,2,115,N,경기도 안양시 호계3동 971-1,951780.982196,1.930038e+06,126.955433,37.368156
1,안양시,HHI브라운빌3차아파트,호계2동,940,2007-07-27,15,2,149,N,경기도 안양시 호계2동 940,951648.800973,1.930982e+06,126.953879,37.376652
2,안양시,LD범계마을아파트,호계2동,923-30,1999-01-13,17,1,113,N,경기도 안양시 호계2동 923-30,951221.572581,1.931673e+06,126.949008,37.382859
3,안양시,LG빌리지,석수2동,484,2001-05-19,10,34,1872,Y,경기도 안양시 석수2동 484,947025.809511,1.936663e+06,126.901258,37.427608
4,안양시,강남6차아파트,비산2동,566-5,1984-10-25,6,1,54,N,경기도 안양시 비산2동 566-5,950089.066159,1.933251e+06,126.936109,37.397025


In [7]:
#7) 버스 data load  
path = '../002_data_dir/003_search_data/k_bus_info.csv'
bus_line_df = pd.read_csv(path)
bus_line_df.rename(columns={'경도':'lng','위도':'lat'},inplace = True)
bus_line_df.head()

,정류장번호,정류장명,lat,lng,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명
0,GGB176500010,광주유스퀘어(종합터미널),35.160517,126.881217,2.020000e+13,0,31250,광주시,경기도
1,GGB179501240,광주비아정류소,35.225400,126.826317,2.020000e+13,0,31250,광주시,경기도
2,GGB200000001,하광교느티나무,37.317600,127.016900,2.020000e+13,1165,31010,수원시,경기도
3,GGB200000002,하광교,37.315933,127.019167,2.020000e+13,1173,31010,수원시,경기도
4,GGB200000003,비석거리입구,37.313233,127.021017,2.020000e+13,1180,31010,수원시,경기도


In [8]:
#8) 환승역 data load  
path = '../002_data_dir/003_search_data/subway_transfer.csv'
subway_transfer_df = pd.read_csv(path)
subway_transfer_df.head()

,철도운영기관명,선명,역명,지번주소,도로명주소,환승,x,y,lng,lat
0,코레일,1호선,가산디지털단지,서울특별시 금천구 가산동 468-4,서울특별시 금천구 벚꽃로 309,y,945373.187085,1.942741e+06,126.882129,37.482290
1,코레일,1호선,구로,서울특별시 구로구 구로동 585-3,서울특별시 구로구 구로중앙로 174,y,945340.922821,1.944965e+06,126.881599,37.502338
2,코레일,1호선,금정,경기도 군포시 금정동 613 금정역,경기도 군포시 군포로 750(금정동),y,950716.653443,1.930504e+06,126.943382,37.372300
3,코레일,1호선,금천구청,서울특별시 금천구 시흥대로63길 91,서울특별시 금천구 시흥대로63길 91,y,946420.134066,1.939861e+06,126.894180,37.456394
4,코레일,1호선,노량진,서울특별시 동작구 노량진동 67-2,서울특별시 동작구 노량진로 151,y,950777.519659,1.946243e+06,126.943019,37.514163


In [9]:
#9) 시각화  ... base map 생성  

center = [37.541, 126.986]

map = folium.Map(
    location=center,
    width=1000, 
    height=800,
    zoom_start= 10
)


In [10]:
#9) 시각화  ... 경기도 경계 표시
k_geo_data_path = '../002_data_dir/001_map_data/kyungki_region.json'
k_geo_data = json.load(
    open(k_geo_data_path, encoding="utf-8"))

In [11]:
#9) 시각화  ... 20-30 밀도 data 
k_pop_df = pd.read_csv("../002_data_dir/003_search_data/20_30_map_temp2.csv")
k_pop_df['adm_cd'] = k_pop_df['adm_cd'].astype(str)
k_pop_df.head()

,adm_cd,adm_nm,pop_tot,pop15_39,pop40_59,pop60
0,41111,수원시 장안구,33370,15902,4991,12477
1,41113,수원시 권선구,45190,21534,6758,16897
2,41115,수원시 팔달구,23669,11279,3540,8850
3,41117,수원시 영통구,44343,21130,6632,16580
4,41131,성남시 수정구,29113,11721,4506,12885


In [12]:
#9) 시각화  ... 20-30 밀도 시각화
folium.Choropleth(
    geo_data=k_geo_data,
    name="경기도 20-30대",
    data=k_pop_df,
    columns=["adm_cd", "pop15_39"],
    key_on="feature.properties.ADM_SECT_C",
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='white', 
    nan_fill_color='blue',
    line_weight=0,
    highlight=False, 
    smooth_factor=1.0,
    legend_name="인구수"
).add_to(map)

In [13]:
#9) 시각화 ... 레이어 적용을 위한 그룹 만들기
# # 대표 그룹 만들기
fg = folium.FeatureGroup(name="groups")
map.add_child(fg)

# 쏘카 그룹 만들기
socar = plugins.FeatureGroupSubGroup(fg, "socar_zone")
map.add_child(socar)

# 행복주택 그룹 만들기
happy = plugins.FeatureGroupSubGroup(fg, "happy house")
map.add_child(happy)

# 대학교 그룹 만들기
university = plugins.FeatureGroupSubGroup(fg, "university")
map.add_child(university)

# 아파트 그룹 만들기
k_apt = plugins.FeatureGroupSubGroup(fg, "APT")
map.add_child(k_apt)

# 지하철 그룹 만들기
subway = plugins.FeatureGroupSubGroup(fg, "subway")
map.add_child(subway)

# 환승역 그룹 만들기 
transfer = plugins.FeatureGroupSubGroup(fg, "transfer")
map.add_child(transfer)

# 버스 그룹 만들기
bus = plugins.FeatureGroupSubGroup(fg, "bus")
map.add_child(bus)

# 레이어 콘트롤 박스 만들기
folium.LayerControl(collapsed=False).add_to(map)

In [14]:
#9) 시각화 ...쏘카존 
mg1 = MarkerCluster()
for _, row in k_socar_zone_df.iterrows():
    mg1.add_child(    
        folium.Marker(location = [row['lat'], row['lng']],
               popup=row['zone_name'],
               icon=folium.Icon(color="cadetblue", icon="car", prefix="fa")
              )
    )

mg1.add_to(socar)

In [15]:
#9) 시각화 ...행복주택 
mg2 = MarkerCluster()
for _, row in happy_house_df.iterrows():
    mg2.add_child(    
        folium.Marker(location = [row['lat'], row['lng']],
               popup=row['단지'],
               icon=folium.Icon(color='red',icon='home', prefix="fa")
              )
    )
    
mg2.add_to(happy)

In [16]:
#9) 시각화 ...대학교
mg3 = MarkerCluster()
for _, row in k_university_df.iterrows():
    mg3.add_child(    
        folium.Marker(location = [row['lat'], row['lng']],
               popup=row['학교명'],
               icon=folium.Icon(color='green',icon='star',prefix="fa")
              )
    )

mg3.add_to(university)

In [17]:
#9) 시각화 ...아파트
mg4 = MarkerCluster()
for _, row in k_apt_df.iterrows():
    mg4.add_child(    
        folium.Marker(location = [row['lat'], row['lng']],
               popup=row['공동주택명정보'],
               icon=folium.Icon(color='orange',icon='building',prefix="fa")
              )
    )

mg4.add_to(k_apt)

In [18]:
#9) 시각화 ...지하철
mg5 = MarkerCluster()
for _, row in k_subway_info_df.iterrows():
    mg5.add_child(    
        folium.Marker(location = [row['lat'], row['lng']],
               popup=row['역명'],
               icon=folium.Icon(color='gray',icon='train',prefix="fa")
              )
    )

mg5.add_to(subway)

In [19]:
#9) 시각화 ...버스
mg6 = MarkerCluster()
for _, row in bus_line_df.iterrows(): 
    mg6.add_child(    
        folium.Marker(location = [row['lat'], row['lng']],
               popup=row['정류장명'],
               icon=folium.Icon(color='lightgray',icon='van-shuttle',prefix="fa")
               
              )
    )

mg6.add_to(bus)

In [20]:
#9) 시각화 ...환승역
mg7 = MarkerCluster()
for _, row in subway_transfer_df.iterrows():
    mg7.add_child(    
        folium.Marker(location = [row['lat'], row['lng']],
               popup=row['역명'],
               icon=folium.Icon(color='purple',icon='flag',prefix="fa")
              )
    )

mg7.add_to(transfer)

In [21]:
# #9) 시각화 ...total_map
# map


In [22]:
# #10) map save
# map.save("../003_map_xml/230203_combine_7group_k20man.html")